In [ ]:
import numpy as np
import pandas as pd
import warnings
import time
from datetime import datetime

from sklearn.impute import KNNImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.linear_model import LinearRegression 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import accuracy_score

from sklearn.metrics import precision_score, confusion_matrix, recall_score, f1_score, classification_report, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

import joblib
from collections import Counter
from sklearn.utils import resample
from sklearn.datasets import make_classification

import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn import metrics
from xgboost.sklearn import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import learning_curve
import pickle
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier


import warnings
warnings.filterwarnings("ignore", category=Warning)

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
from scipy.stats import norm


#显示所有列，把行显示设置成最大
pd.set_option('display.max_columns', 500)

In [ ]:
def find_missing(data):
    missing_num = data.isna().sum(axis=0).sort_values(ascending=False)
    missing_prop = missing_num/float(len(data)) 
    drop_index = missing_prop[missing_prop>0.3].index.tolist() 
    return drop_index

In [ ]:
from sklearn.feature_selection import VarianceThreshold
variance_1 = VarianceThreshold(threshold = (0.90*(1-0.90))) 

In [33]:
feature3['中性粒细胞/淋巴细胞计数(NLR)'] = feature3['中性粒细胞/淋巴细胞计数(NLR)'].replace(np.inf,np.nan)
feature3['血小板/淋巴细胞比值(PLR)'] = feature3['血小板/淋巴细胞比值(PLR)'].replace(np.inf,np.nan)
feature3['淋巴细胞/单核细胞比值(LMR)'] = feature3['淋巴细胞/单核细胞比值(LMR)'].replace(np.inf,np.nan)
feature3['血小板/白蛋白比值(PAR)'] = feature3['血小板/白蛋白比值(PAR)'].replace(np.inf,np.nan)

In [ ]:
param_dist = {
    'n_neighbors':  np.arange(40,500,20),
    'weights': ['uniform', 'distance'],
    }
    

imputer_knn = KNNImputer()

random_search = RandomizedSearchCV(imputer_knn, param_distributions=param_dist, cv=5, scoring='roc_auc',n_iter=10) 

random_search.fit(feature3[continuous_features3] , y['是否在28天内死亡'])

best_params = random_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
imputer_best_knn = KNNImputer(**best_params)
features_imputed_knn = imputer_best_knn.fit_transform(feature3[continuous_features3])

df_imputed_knn = pd.DataFrame(features_imputed_knn, columns=feature3[continuous_features3].columns)
df_imputed_knn = pd.concat([df_imputed_knn, feature3[categorical_features3]],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_imputed_knn, y['是否在28天内死亡'], test_size=0.3)

rf = RandomForestClassifier()s
rf.fit(X_train, y_train)

y_pred_proba = rf.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_pred_proba)
print(f'best_AUC: {auc:.4f}')

In [ ]:
imputer_lr_mi = IterativeImputer(estimator=LinearRegression())

imputed_data_lr_mi = imputer_lr_mi.fit_transform(feature3[continuous_features3])

imputed_df_lr_mi = pd.DataFrame(imputed_data_lr_mi, columns=feature3[continuous_features3].columns)
imputed_df_lr_mi = pd.concat([imputed_df_lr_mi,feature3[categorical_features3]],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imputed_df_lr_mi, y['是否在28天内死亡'], test_size=0.3)

rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train, y_train)

y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_pred_proba)

print(f'best_AUC: {auc:.4f}')

In [ ]:
imputer_knn_mi = IterativeImputer(estimator=KNeighborsRegressor())

imputed_data_knn_mi = imputer_knn_mi.fit_transform(feature3[continuous_features3].astype(float))

imputed_df_knn_mi = pd.DataFrame(imputed_data_knn_mi, columns=feature3[continuous_features3].columns)
imputed_df_knn_mi = pd.concat([imputed_df_knn_mi,feature3[categorical_features3]],axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imputed_df_knn_mi, y['是否在28天内死亡'], test_size=0.3)

rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train, y_train)

y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_pred_proba)
print(f'best_AUC: {auc:.4f}')

In [54]:
imputer_rr = SimpleImputer(strategy='mean')
df_imputed_rr = pd.DataFrame(imputer_rr.fit_transform(feature3[continuous_features3]),columns=feature3[continuous_features3].columns)

In [ ]:
for target_col in feature3[continuous_features3].columns:
    X = df_imputed_rr.drop(target_col, axis=1).values  
    y_1 = df_imputed_rr[target_col]  

    ridge_model = Ridge()  
    ridge_model.fit(X, y_1)
    
    y_pred = ridge_model.predict(X)
    
    df_imputed_rr[target_col] = y_pred

In [ ]:
imputed_df_rr = pd.concat([df_imputed_rr,feature3[categorical_features3]],axis=1)

X_train, X_test, y_train, y_test = train_test_split(imputed_df_rr, y['是否在28天内死亡'], test_size=0.3)

rf_classifier = RandomForestClassifier()

rf_classifier.fit(X_train, y_train)

y_pred_proba = rf_classifier.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_pred_proba)
print(f'best_AUC: {auc:.4f}')

In [ ]:
corr_matrix = feature4.corr().abs()

high_corr_vars = np.where(corr_matrix >= 0.7)
high_corr_vars = [(corr_matrix.index[x], corr_matrix.columns[y]) for x, y in zip(*high_corr_vars) if x != y and x < y]
print(high_corr_vars)

In [ ]:
def calculate_entropy(series):
    value_counts = series.value_counts(normalize=True, dropna=False)
    entropy = -(value_counts * np.log2(value_counts + 1e-10)).sum()
    return entropy

feature_entropies = model_data.drop(['是否在28天内死亡','就诊标识（医渡云计算）'],axis=1).apply(calculate_entropy)


In [98]:
entr_result = pd.DataFrame(feature_entropies).reset_index()
entr_result.columns=['变量','熵值']
entr_result = entr_result.sort_values(by='熵值',ascending=False)

In [ ]:
auc_scores = []

for i in np.arange(2,52,2):  
    top_variables = entr_result['变量'].head(i).tolist()
    X = model_data[top_variables]
    y = model_data['是否在28天内死亡']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    rf_model = RandomForestClassifier()
    rf_model.fit(X_train, y_train)

    y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_pred_proba)

    auc_scores.append(auc)

In [ ]:
auc_result = pd.DataFrame({'变量个数':np.arange(2,52,2),'AUC':auc_scores,})

In [ ]:
fig, ax = plt.subplots()

plt.rcParams['font.family'] = 'Times New Roman'

ax.plot(auc_result['变量个数'], auc_result['AUC'], marker='o', linestyle='-', color='b')

ax.set_xlabel('variables_number')
ax.set_ylabel('AUC_score')

ax.set_ylim(0.4, 1.0)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.axhline(y=0.6, color='gray', linestyle='--', linewidth=0.5) 
ax.axhline(y=0.7, color='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=0.8, color='gray', linestyle='--', linewidth=0.5)
ax.axhline(y=0.9, color='gray', linestyle='--', linewidth=0.5)

plt.scatter(34, auc_result[auc_result['变量个数']==34]['AUC'], color='blue', marker='*',s=200)
plt.text(34, auc_result[auc_result['变量个数']==34]['AUC']+0.03, '(34, 0.8410)', fontsize=12, 
             ha='center', va='bottom', color='black')

plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize=(25, 25))

ax = sns.heatmap(corr_matrix_rename, cmap='Greens', annot=False, xticklabels=1, yticklabels=1) 

plt.rcParams['font.sans-serif'] = 'times new roman'
plt.xticks(fontsize=24) 
plt.yticks(fontsize=24)  

cbar = ax.collections[0].colorbar

cbar.ax.tick_params(labelsize=24) 
plt.show()

In [ ]:
scaler_important = MinMaxScaler()
X_train_scaler = scaler_important.fit_transform(X_model)

In [ ]:
smote = SMOTE(sampling_strategy=1,k_neighbors=20)
X_resampled, y_resampled = smote.fit_resample(X_train_scaler, y_model)

In [ ]:
model_tree = DecisionTreeClassifier()
param_grid_tree = {
    'criterion':['entropy','gini'],   
    'max_depth':np.arange(10,300,10),      
    'max_features':['sqrt' , 'log2'], 
    'max_leaf_nodes':[10,20,30,40,50,60,70,80,90,100],  
    'min_samples_split':[10,20,30,40,50,60,70,80,90,100],
    'min_samples_leaf':[10,20,30,40,50,60,70,80,90,100],
    'splitter':['best' ,'random']
}

random_search_tree = RandomizedSearchCV(model_tree, param_grid_tree,scoring='roc_auc',cv=5)
random_search_tree.fit(X_resampled, y_resampled)

best_criterion = random_search_tree.best_params_['criterion']
best_max_depth = random_search_tree.best_params_['max_depth']
best_max_features = random_search_tree.best_params_['max_features']
best_max_leaf_nodes = random_search_tree.best_params_['max_leaf_nodes']
best_min_samples_split = random_search_tree.best_params_['min_samples_split']
best_splitter = random_search_tree.best_params_['splitter']
best_min_samples_leaf = random_search_tree.best_params_['min_samples_leaf']

print('best_criterion：',best_criterion)
print('best_max_depth：',best_max_depth)
print('best_max_features：',best_max_features)
print('best_max_leaf_nodes：',best_max_leaf_nodes)
print('best_min_samples_split：',best_min_samples_split)
print('best_splitter：',best_splitter)
print('best_min_samples_leaf：',best_min_samples_leaf)

In [ ]:
model_tree_result = DecisionTreeClassifier(criterion=best_criterion,
                                              max_depth=best_max_depth, max_features=best_max_features ,
                                              max_leaf_nodes = best_max_leaf_nodes, min_samples_leaf = best_min_samples_leaf,
                                              splitter = best_splitter,
                                              min_samples_split = best_min_samples_split)
model_tree_result.fit(X_resampled, y_resampled)

In [ ]:
tree_y_pred_auc = model_tree_result.predict(X_resampled)
y_probas_tree_auc = model_tree_result.predict_proba(X_resampled)

y_true = np.array(y_resampled.copy()) 
y_pred = np.array(tree_y_pred_auc.copy()) 
y_probas = np.array(y_probas_tree_auc)

auc_scores = []
f1_scores = []
acc_scores = []
pre_scores = []
sen_scores = []
spe_scores = []
fprs = []
tprs = []

for i in range(1000):
    sample_indices = np.random.choice(len(y_true), size=len(y_true), replace=True)
    y_true_sample = y_true[sample_indices]
    y_pred_sample = y_pred[sample_indices]
    y_probas_sample = y_probas[sample_indices]

    fpr, tpr, _ = roc_curve(y_true_sample, y_probas_sample[:, 1])
    auc_scores.append(auc(fpr, tpr))
    fprs.append(fpr)
    tprs.append(tpr)
 
    f1_scores.append(f1_score(y_true_sample, y_pred_sample))
    acc_scores.append(accuracy_score(y_true_sample, y_pred_sample))
    pre_scores.append(precision_score(y_true_sample, y_pred_sample))
    sen_scores.append(recall_score(y_true_sample, y_pred_sample))
    tn, fp, fn, tp = confusion_matrix(y_true_sample, y_pred_sample).ravel()
    spe_scores.append(tn / (tn + fp))

auc_median = np.median(auc_scores)
auc_lower = np.percentile(auc_scores, 2.5)
auc_upper = np.percentile(auc_scores, 97.5)

f1_median = np.median(f1_scores)
f1_lower = np.percentile(f1_scores, 2.5)
f1_upper = np.percentile(f1_scores, 97.5)

acc_median = np.median(acc_scores)
acc_lower = np.percentile(acc_scores, 2.5)
acc_upper = np.percentile(acc_scores, 97.5)

pre_median = np.median(pre_scores)
pre_lower = np.percentile(pre_scores, 2.5)
pre_upper = np.percentile(pre_scores, 97.5)

sen_median = np.median(sen_scores)
sen_lower = np.percentile(sen_scores, 2.5)
sen_upper = np.percentile(sen_scores, 97.5)

spe_median = np.median(spe_scores)
spe_lower = np.percentile(spe_scores, 2.5)
spe_upper = np.percentile(spe_scores, 97.5)

print(f"AUC: Median = {auc_median}, 95% CI = [{auc_lower}, {auc_upper}]")
print(f"F1: Median = {f1_median}, 95% CI = [{f1_lower}, {f1_upper}]")
print(f"ACC: Median = {acc_median}, 95% CI = [{acc_lower}, {acc_upper}]")
print(f"PRE: Median = {pre_median}, 95% CI = [{pre_lower}, {pre_upper}]")
print(f"SEN: Median = {sen_median}, 95% CI = [{sen_lower}, {sen_upper}]")
print(f"SPE: Median = {spe_median}, 95% CI = [{spe_lower}, {spe_upper}]")

In [ ]:
model_rf = RandomForestClassifier()

param_grid_rf = {
    'n_estimators':np.arange(20,100,5),  
    'criterion':['gini','entropy'], 
    'max_depth':[10,20,30,40,50,60,70,80,90,100],  
    'min_samples_leaf':[10,20,30,40,50,60,70,80,90,100],       
    'min_samples_split':[10,20,30,40,50,60,70,80,90,100],   
    'max_features':['sqrt','log2']  
}

random_search_rf = RandomizedSearchCV(model_rf, param_grid_rf,scoring='roc_auc',cv=5)
random_search_rf.fit(X_resampled, y_resampled)

best_n_estimators = random_search_rf.best_params_['n_estimators']
best_criterion = random_search_rf.best_params_['criterion']
best_max_depth = random_search_rf.best_params_['max_depth']
best_min_samples_leaf = random_search_rf.best_params_['min_samples_leaf']
best_min_samples_split = random_search_rf.best_params_['min_samples_split']
best_max_features = random_search_rf.best_params_['max_features']

print('best_n_estimators：',best_n_estimators)
print('best_criterion ：',best_criterion)
print('best_max_depth：',best_max_depth)
print('best_min_samples_leaf：',best_min_samples_leaf)
print('best_min_samples_split：',best_min_samples_split)
print('best_max_features：',best_max_features)

In [ ]:
model_rf_result = RandomForestClassifier(  n_estimators=best_n_estimators, criterion=best_criterion,
                                           max_depth=best_max_depth, min_samples_leaf=best_min_samples_leaf,
                                           min_samples_split=best_min_samples_split, max_features=best_max_features)
model_rf_result.fit(X_resampled, y_resampled)

In [ ]:
rf_y_pred_auc = model_rf_result.predict(X_resampled)
y_probas_rf_auc = model_rf_result.predict_proba(X_resampled)

y_true = np.array(y_resampled.copy()) 
y_pred = np.array(rf_y_pred_auc.copy()) 
y_probas = np.array(y_probas_rf_auc)

auc_scores = []
f1_scores = []
acc_scores = []
pre_scores = []
sen_scores = []
spe_scores = []
fprs = []
tprs = []

for i in range(1000):
    sample_indices = np.random.choice(len(y_true), size=len(y_true), replace=True)
    y_true_sample = y_true[sample_indices]
    y_pred_sample = y_pred[sample_indices]
    y_probas_sample = y_probas[sample_indices]
    
    fpr, tpr, _ = roc_curve(y_true_sample, y_probas_sample[:, 1])
    auc_scores.append(auc(fpr, tpr))
    fprs.append(fpr)
    tprs.append(tpr)
    
    f1_scores.append(f1_score(y_true_sample, y_pred_sample))
    acc_scores.append(accuracy_score(y_true_sample, y_pred_sample))
    pre_scores.append(precision_score(y_true_sample, y_pred_sample))
    sen_scores.append(recall_score(y_true_sample, y_pred_sample))
    tn, fp, fn, tp = confusion_matrix(y_true_sample, y_pred_sample).ravel()
    spe_scores.append(tn / (tn + fp))

auc_median = np.median(auc_scores)
auc_lower = np.percentile(auc_scores, 2.5)
auc_upper = np.percentile(auc_scores, 97.5)

f1_median = np.median(f1_scores)
f1_lower = np.percentile(f1_scores, 2.5)
f1_upper = np.percentile(f1_scores, 97.5)

acc_median = np.median(acc_scores)
acc_lower = np.percentile(acc_scores, 2.5)
acc_upper = np.percentile(acc_scores, 97.5)

pre_median = np.median(pre_scores)
pre_lower = np.percentile(pre_scores, 2.5)
pre_upper = np.percentile(pre_scores, 97.5)

sen_median = np.median(sen_scores)
sen_lower = np.percentile(sen_scores, 2.5)
sen_upper = np.percentile(sen_scores, 97.5)

spe_median = np.median(spe_scores)
spe_lower = np.percentile(spe_scores, 2.5)
spe_upper = np.percentile(spe_scores, 97.5)

print(f"AUC: Median = {auc_median}, 95% CI = [{auc_lower}, {auc_upper}]")
print(f"F1: Median = {f1_median}, 95% CI = [{f1_lower}, {f1_upper}]")
print(f"ACC: Median = {acc_median}, 95% CI = [{acc_lower}, {acc_upper}]")
print(f"PRE: Median = {pre_median}, 95% CI = [{pre_lower}, {pre_upper}]")
print(f"SEN: Median = {sen_median}, 95% CI = [{sen_lower}, {sen_upper}]")
print(f"SPE: Median = {spe_median}, 95% CI = [{spe_lower}, {spe_upper}]")

In [ ]:
model_xgb = XGBClassifier()

param_grid_xgb = {
    'max_depth': np.arange(20,50,5),
    'min_child_weight': np.arange(5,20,1),
    'lambda': np.arange(10,50,5),
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'eta': np.arange(0.01,0.1,0.01),
    'gamma': [0.001,0.005,0.01,0.05,0.1],
    'n_estimators': np.arange(20,50,5)
}

random_search_xgb = RandomizedSearchCV(model_xgb, param_grid_xgb, scoring='roc_auc',cv=5)
random_search_xgb.fit(X_resampled, y_resampled)

best_n_estimators = random_search_xgb.best_params_['n_estimators']
best_max_depth = random_search_xgb.best_params_['max_depth']
best_min_child_weight = random_search_xgb.best_params_['min_child_weight']
best_subsample = random_search_xgb.best_params_['subsample']
best_colsample_bytree = random_search_xgb.best_params_['colsample_bytree']
best_gamma = random_search_xgb.best_params_['gamma']
best_lambda = random_search_xgb.best_params_['lambda']
best_eta = random_search_xgb.best_params_['eta']

print('best_n_estimators: ', random_search_xgb.best_params_['n_estimators'])
print('best_max_depth: ', random_search_xgb.best_params_['max_depth'])
print('best_min_child_weight: ', random_search_xgb.best_params_['min_child_weight'])
print('best_subsample: ', random_search_xgb.best_params_['subsample'])
print('best_colsample_bytree: ', random_search_xgb.best_params_['colsample_bytree'])
print('best_gamma: ', random_search_xgb.best_params_['gamma'])
print('best_lambda: ', random_search_xgb.best_params_['lambda'])
print('best_eta: ', random_search_xgb.best_params_['eta'])

In [ ]:
model_xgb_result = XGBClassifier(n_estimators=best_n_estimators,
                                    max_depth=best_max_depth,
                                    min_child_weight=best_min_child_weight,
                                    subsample=best_subsample,
                                    colsample_bytree=best_colsample_bytree,
                                    gamma=best_gamma,
                                    reg_lambda=best_lambda,
                                    eta=best_eta)
model_xgb_result.fit(X_resampled, y_resampled)

In [ ]:
xgb_y_pred_auc = model_xgb_result.predict(X_resampled)
y_probas_xgb_auc = model_xgb_result.predict_proba(X_resampled)

y_true = np.array(y_resampled.copy()) 
y_pred = np.array(xgb_y_pred_auc.copy()) 
y_probas = np.array(y_probas_xgb_auc)

auc_scores = []
f1_scores = []
acc_scores = []
pre_scores = []
sen_scores = []
spe_scores = []
fprs = []
tprs = []

for i in range(1000):

    sample_indices = np.random.choice(len(y_true), size=len(y_true), replace=True)
    y_true_sample = y_true[sample_indices]
    y_pred_sample = y_pred[sample_indices]
    y_probas_sample = y_probas[sample_indices]

    fpr, tpr, _ = roc_curve(y_true_sample, y_probas_sample[:, 1])
    auc_scores.append(auc(fpr, tpr))
    fprs.append(fpr)
    tprs.append(tpr)
    
    f1_scores.append(f1_score(y_true_sample, y_pred_sample))
    acc_scores.append(accuracy_score(y_true_sample, y_pred_sample))
    pre_scores.append(precision_score(y_true_sample, y_pred_sample))
    sen_scores.append(recall_score(y_true_sample, y_pred_sample))
    tn, fp, fn, tp = confusion_matrix(y_true_sample, y_pred_sample).ravel()
    spe_scores.append(tn / (tn + fp))

auc_median = np.median(auc_scores)
auc_lower = np.percentile(auc_scores, 2.5)
auc_upper = np.percentile(auc_scores, 97.5)

f1_median = np.median(f1_scores)
f1_lower = np.percentile(f1_scores, 2.5)
f1_upper = np.percentile(f1_scores, 97.5)

acc_median = np.median(acc_scores)
acc_lower = np.percentile(acc_scores, 2.5)
acc_upper = np.percentile(acc_scores, 97.5)

pre_median = np.median(pre_scores)
pre_lower = np.percentile(pre_scores, 2.5)
pre_upper = np.percentile(pre_scores, 97.5)

sen_median = np.median(sen_scores)
sen_lower = np.percentile(sen_scores, 2.5)
sen_upper = np.percentile(sen_scores, 97.5)

spe_median = np.median(spe_scores)
spe_lower = np.percentile(spe_scores, 2.5)
spe_upper = np.percentile(spe_scores, 97.5)

print(f"AUC: Median = {auc_median}, 95% CI = [{auc_lower}, {auc_upper}]")
print(f"F1: Median = {f1_median}, 95% CI = [{f1_lower}, {f1_upper}]")
print(f"ACC: Median = {acc_median}, 95% CI = [{acc_lower}, {acc_upper}]")
print(f"PRE: Median = {pre_median}, 95% CI = [{pre_lower}, {pre_upper}]")
print(f"SEN: Median = {sen_median}, 95% CI = [{sen_lower}, {sen_upper}]")
print(f"SPE: Median = {spe_median}, 95% CI = [{spe_lower}, {spe_upper}]")

In [ ]:
model_lgb = LGBMClassifier()

param_grid_lgb = {

    'boosting_type': ['gbdt','goss','dart','rf'],
    'num_leaves': np.arange(5,50,5),
    'max_depth': np.arange(10,50,5),
    'learning_rate': np.arange(0.001,0.05,0.01),
    'n_estimators': np.arange(10,50,5),
    'subsample': np.arange(0.1,1,0.1),
    'colsample_bytree': np.arange(0.1,1,0.1),
    'reg_alpha': [0.01, 0.05, 0.1, 0.5 ,1],
    'verbosity': [-1]
}

random_search_lgb = RandomizedSearchCV(model_lgb, param_grid_lgb,scoring='roc_auc',cv=5)
random_search_lgb.fit(X_resampled, y_resampled)

best_boosting_type = random_search_lgb.best_params_['boosting_type']
best_num_leaves = random_search_lgb.best_params_['num_leaves']
best_max_depth = random_search_lgb.best_params_['max_depth']
best_learning_rate = random_search_lgb.best_params_['learning_rate']
best_n_estimators = random_search_lgb.best_params_['n_estimators']
best_subsample = random_search_lgb.best_params_['subsample']
best_colsample_bytree = random_search_lgb.best_params_['colsample_bytree']
best_reg_alpha = random_search_lgb.best_params_['reg_alpha']

print('best_boosting_type:',best_boosting_type)
print('best_num_leaves:',best_num_leaves)
print('best_max_depth:',best_max_depth)
print('best_learning_rate:',best_learning_rate)
print('best_n_estimators:',best_n_estimators)
print('best_subsample:',best_subsample)
print('best_colsample_bytree:',best_colsample_bytree)
print('best_reg_alpha:',best_reg_alpha)

In [ ]:
model_lgb_result = LGBMClassifier(boosting_type=best_boosting_type,
                                     num_leaves=best_num_leaves,
                                     max_depth=best_max_depth,
                                     learning_rate=best_learning_rate,
                                     n_estimators=best_n_estimators,
                                     subsample=best_subsample,
                                     colsample_bytree=best_colsample_bytree,
                                     reg_alpha=best_reg_alpha, verbosity=-1)

model_lgb_result.fit(X_resampled,y_resampled)

In [ ]:
lgb_y_pred_auc = model_lgb_result.predict(X_resampled)
y_probas_lgb_auc = model_lgb_result.predict_proba(X_resampled)

y_true = np.array(y_resampled.copy()) 
y_pred = np.array(lgb_y_pred_auc.copy()) 
y_probas = np.array(y_probas_lgb_auc)

auc_scores = []
f1_scores = []
acc_scores = []
pre_scores = []
sen_scores = []
spe_scores = []
fprs = []
tprs = []

for i in range(1000):
    sample_indices = np.random.choice(len(y_true), size=len(y_true), replace=True)
    y_true_sample = y_true[sample_indices]
    y_pred_sample = y_pred[sample_indices]
    y_probas_sample = y_probas[sample_indices]

    fpr, tpr, _ = roc_curve(y_true_sample, y_probas_sample[:, 1])
    auc_scores.append(auc(fpr, tpr))
    fprs.append(fpr)
    tprs.append(tpr)
    
    f1_scores.append(f1_score(y_true_sample, y_pred_sample))
    acc_scores.append(accuracy_score(y_true_sample, y_pred_sample))
    pre_scores.append(precision_score(y_true_sample, y_pred_sample))
    sen_scores.append(recall_score(y_true_sample, y_pred_sample))
    tn, fp, fn, tp = confusion_matrix(y_true_sample, y_pred_sample).ravel()
    spe_scores.append(tn / (tn + fp))

auc_median = np.median(auc_scores)
auc_lower = np.percentile(auc_scores, 2.5)
auc_upper = np.percentile(auc_scores, 97.5)

f1_median = np.median(f1_scores)
f1_lower = np.percentile(f1_scores, 2.5)
f1_upper = np.percentile(f1_scores, 97.5)

acc_median = np.median(acc_scores)
acc_lower = np.percentile(acc_scores, 2.5)
acc_upper = np.percentile(acc_scores, 97.5)

pre_median = np.median(pre_scores)
pre_lower = np.percentile(pre_scores, 2.5)
pre_upper = np.percentile(pre_scores, 97.5)

sen_median = np.median(sen_scores)
sen_lower = np.percentile(sen_scores, 2.5)
sen_upper = np.percentile(sen_scores, 97.5)

spe_median = np.median(spe_scores)
spe_lower = np.percentile(spe_scores, 2.5)
spe_upper = np.percentile(spe_scores, 97.5)

print(f"AUC: Median = {auc_median}, 95% CI = [{auc_lower}, {auc_upper}]")
print(f"F1: Median = {f1_median}, 95% CI = [{f1_lower}, {f1_upper}]")
print(f"ACC: Median = {acc_median}, 95% CI = [{acc_lower}, {acc_upper}]")
print(f"PRE: Median = {pre_median}, 95% CI = [{pre_lower}, {pre_upper}]")
print(f"SEN: Median = {sen_median}, 95% CI = [{sen_lower}, {sen_upper}]")
print(f"SPE: Median = {spe_median}, 95% CI = [{spe_lower}, {spe_upper}]")

In [ ]:
svm = SVC()

param_grid = {
    'C': np.arange(0.01,0.5,0.01),          
    'gamma': np.arange(0.01,0.5,0.01),        
    'kernel': ['linear', 'rbf', 'poly'],   
    'degree': np.arange(1,10,1),                  
}

random_search_svm = RandomizedSearchCV(svm, param_distributions=param_grid, scoring='roc_auc', cv=5 )
random_search_svm.fit(X_resampled, y_resampled)

best_C = random_search_svm.best_params_['C']
best_gamma = random_search_svm.best_params_['gamma']
best_kernel = random_search_svm.best_params_['kernel']
best_degree = random_search_svm.best_params_['degree']


print('best_C:',best_C)
print('best_gamma:',best_gamma)
print('best_kernel:',best_kernel)
print('best_degree:',best_degree)

In [ ]:
model_svm_result = SVC (C=best_C,
                                   gamma = best_gamma,
                                   kernel = best_kernel,
                                   degree = best_degree,probability=True )
model_svm_result.fit(X_resampled, y_resampled)

In [ ]:
svm_y_pred_auc = model_svm_result.predict(X_resampled)
y_probas_svm_auc = model_svm_result.predict_proba(X_resampled)

y_true = np.array(y_resampled.copy()) 
y_pred = np.array(svm_y_pred_auc.copy()) 
y_probas = np.array(y_probas_svm_auc)

auc_scores = []
f1_scores = []
acc_scores = []
pre_scores = []
sen_scores = []
spe_scores = []
fprs = []
tprs = []

for i in range(1000):
    sample_indices = np.random.choice(len(y_true), size=len(y_true), replace=True)
    y_true_sample = y_true[sample_indices]
    y_pred_sample = y_pred[sample_indices]
    y_probas_sample = y_probas[sample_indices]
    
    fpr, tpr, _ = roc_curve(y_true_sample, y_probas_sample[:, 1])
    auc_scores.append(auc(fpr, tpr))
    fprs.append(fpr)
    tprs.append(tpr)
    
    f1_scores.append(f1_score(y_true_sample, y_pred_sample))
    acc_scores.append(accuracy_score(y_true_sample, y_pred_sample))
    pre_scores.append(precision_score(y_true_sample, y_pred_sample))
    sen_scores.append(recall_score(y_true_sample, y_pred_sample))
    tn, fp, fn, tp = confusion_matrix(y_true_sample, y_pred_sample).ravel()
    spe_scores.append(tn / (tn + fp))

auc_median = np.median(auc_scores)
auc_lower = np.percentile(auc_scores, 2.5)
auc_upper = np.percentile(auc_scores, 97.5)

f1_median = np.median(f1_scores)
f1_lower = np.percentile(f1_scores, 2.5)
f1_upper = np.percentile(f1_scores, 97.5)

acc_median = np.median(acc_scores)
acc_lower = np.percentile(acc_scores, 2.5)
acc_upper = np.percentile(acc_scores, 97.5)

pre_median = np.median(pre_scores)
pre_lower = np.percentile(pre_scores, 2.5)
pre_upper = np.percentile(pre_scores, 97.5)

sen_median = np.median(sen_scores)
sen_lower = np.percentile(sen_scores, 2.5)
sen_upper = np.percentile(sen_scores, 97.5)

spe_median = np.median(spe_scores)
spe_lower = np.percentile(spe_scores, 2.5)
spe_upper = np.percentile(spe_scores, 97.5)

print(f"AUC: Median = {auc_median}, 95% CI = [{auc_lower}, {auc_upper}]")
print(f"F1: Median = {f1_median}, 95% CI = [{f1_lower}, {f1_upper}]")
print(f"ACC: Median = {acc_median}, 95% CI = [{acc_lower}, {acc_upper}]")
print(f"PRE: Median = {pre_median}, 95% CI = [{pre_lower}, {pre_upper}]")
print(f"SEN: Median = {sen_median}, 95% CI = [{sen_lower}, {sen_upper}]")
print(f"SPE: Median = {spe_median}, 95% CI = [{spe_lower}, {spe_upper}]")

In [ ]:
nb = BernoulliNB()

param_dist = {

    'alpha': np.linspace(0.1, 1, 10), 
    'binarize': [None] + list(np.linspace(0.0, 1.0, 11)) 

}

random_search_nb = RandomizedSearchCV(nb, param_distributions=param_dist, scoring='roc_auc', cv=5)
random_search_nb.fit(X_resampled, y_resampled)

best_alpha = random_search_nb.best_params_['alpha']
best_binarize = random_search_nb.best_params_['binarize']


print('best_alpha:',best_alpha)
print('best_binarize:',best_binarize)

In [ ]:
model_nb_result = BernoulliNB(alpha=best_alpha,binarize = best_binarize)
model_nb_result.fit(X_resampled, y_resampled)

In [ ]:
nb_y_pred_auc = model_nb_result.predict(X_resampled)
y_probas_nb_auc = model_nb_result.predict_proba(X_resampled)

y_true = np.array(y_resampled.copy()) 
y_pred = np.array(nb_y_pred_auc.copy()) 
y_probas = np.array(y_probas_nb_auc)

auc_scores = []
f1_scores = []
acc_scores = []
pre_scores = []
sen_scores = []
spe_scores = []
fprs = []
tprs = []


for i in range(1000):
    sample_indices = np.random.choice(len(y_true), size=len(y_true), replace=True)
    y_true_sample = y_true[sample_indices]
    y_pred_sample = y_pred[sample_indices]
    y_probas_sample = y_probas[sample_indices]
    
    fpr, tpr, _ = roc_curve(y_true_sample, y_probas_sample[:, 1])
    auc_scores.append(auc(fpr, tpr))
    fprs.append(fpr)
    tprs.append(tpr)

    f1_scores.append(f1_score(y_true_sample, y_pred_sample))
    acc_scores.append(accuracy_score(y_true_sample, y_pred_sample))
    pre_scores.append(precision_score(y_true_sample, y_pred_sample))
    sen_scores.append(recall_score(y_true_sample, y_pred_sample))
    tn, fp, fn, tp = confusion_matrix(y_true_sample, y_pred_sample).ravel()
    spe_scores.append(tn / (tn + fp))

auc_median = np.median(auc_scores)
auc_lower = np.percentile(auc_scores, 2.5)
auc_upper = np.percentile(auc_scores, 97.5)

f1_median = np.median(f1_scores)
f1_lower = np.percentile(f1_scores, 2.5)
f1_upper = np.percentile(f1_scores, 97.5)

acc_median = np.median(acc_scores)
acc_lower = np.percentile(acc_scores, 2.5)
acc_upper = np.percentile(acc_scores, 97.5)

pre_median = np.median(pre_scores)
pre_lower = np.percentile(pre_scores, 2.5)
pre_upper = np.percentile(pre_scores, 97.5)

sen_median = np.median(sen_scores)
sen_lower = np.percentile(sen_scores, 2.5)
sen_upper = np.percentile(sen_scores, 97.5)

spe_median = np.median(spe_scores)
spe_lower = np.percentile(spe_scores, 2.5)
spe_upper = np.percentile(spe_scores, 97.5)

print(f"AUC: Median = {auc_median}, 95% CI = [{auc_lower}, {auc_upper}]")
print(f"F1: Median = {f1_median}, 95% CI = [{f1_lower}, {f1_upper}]")
print(f"ACC: Median = {acc_median}, 95% CI = [{acc_lower}, {acc_upper}]")
print(f"PRE: Median = {pre_median}, 95% CI = [{pre_lower}, {pre_upper}]")
print(f"SEN: Median = {sen_median}, 95% CI = [{sen_lower}, {sen_upper}]")
print(f"SPE: Median = {spe_median}, 95% CI = [{spe_lower}, {spe_upper}]")

In [ ]:
gb_clf = GradientBoostingClassifier()


param_grid = {
    'n_estimators': np.arange(10,100,10),
    'learning_rate': np.arange(0.01,0.1,10),
    'max_depth': np.arange(2,20,1),
    'subsample':[0.4,0.5,0.6,0.7,0.8],
    'min_samples_split':np.arange(2,20,2),
    'min_samples_leaf':np.arange(20,50,5),
    'max_features':['sqrt','log2']
    }


random_search_gbdt = RandomizedSearchCV(estimator = gb_clf, param_distributions=param_grid, scoring='roc_auc', cv=5)
random_search_gbdt.fit(X_resampled, y_resampled)

best_n_estimators = random_search_gbdt.best_params_['n_estimators']
best_learning_rate = random_search_gbdt.best_params_['learning_rate']
best_max_depth = random_search_gbdt.best_params_['max_depth']
best_subsample = random_search_gbdt.best_params_['subsample']
best_min_samples_split = random_search_gbdt.best_params_['min_samples_split']
best_min_samples_leaf = random_search_gbdt.best_params_['min_samples_leaf']
best_max_features = random_search_gbdt.best_params_['max_features']

print('best_n_estimators:',best_n_estimators)
print('best_learning_rate:',best_learning_rate)
print('best_max_depth:',best_max_depth)
print('best_subsample:',best_subsample)
print('best_min_samples_split:',best_min_samples_split)
print('best_min_samples_leaf:',best_min_samples_leaf)
print('best_max_features:',best_max_features)

In [ ]:
model_gbdt_result = GradientBoostingClassifier(n_estimators = best_n_estimators,
                                                learning_rate = best_learning_rate,
                                                max_depth = best_max_depth,
                                                subsample = best_subsample,
                                                min_samples_split = best_min_samples_split,
                                                min_samples_leaf = best_min_samples_leaf,
                                                max_features = best_max_features)
model_gbdt_result.fit(X_resampled, y_resampled)

In [ ]:
gbdt_y_pred_auc = model_gbdt_result.predict(X_resampled)
y_probas_gbdt_auc = model_gbdt_result.predict_proba(X_resampled)

y_true = np.array(y_resampled.copy()) 
y_pred = np.array(gbdt_y_pred_auc.copy()) 
y_probas = np.array(y_probas_gbdt_auc)

auc_scores = []
f1_scores = []
acc_scores = []
pre_scores = []
sen_scores = []
spe_scores = []
fprs = []
tprs = []

for i in range(1000):

    sample_indices = np.random.choice(len(y_true), size=len(y_true), replace=True)
    y_true_sample = y_true[sample_indices]
    y_pred_sample = y_pred[sample_indices]
    y_probas_sample = y_probas[sample_indices]
    

    fpr, tpr, _ = roc_curve(y_true_sample, y_probas_sample[:, 1])
    auc_scores.append(auc(fpr, tpr))
    fprs.append(fpr)
    tprs.append(tpr)
    

    f1_scores.append(f1_score(y_true_sample, y_pred_sample))
    acc_scores.append(accuracy_score(y_true_sample, y_pred_sample))
    pre_scores.append(precision_score(y_true_sample, y_pred_sample))
    sen_scores.append(recall_score(y_true_sample, y_pred_sample))
    tn, fp, fn, tp = confusion_matrix(y_true_sample, y_pred_sample).ravel()
    spe_scores.append(tn / (tn + fp))

auc_median = np.median(auc_scores)
auc_lower = np.percentile(auc_scores, 2.5)
auc_upper = np.percentile(auc_scores, 97.5)

f1_median = np.median(f1_scores)
f1_lower = np.percentile(f1_scores, 2.5)
f1_upper = np.percentile(f1_scores, 97.5)

acc_median = np.median(acc_scores)
acc_lower = np.percentile(acc_scores, 2.5)
acc_upper = np.percentile(acc_scores, 97.5)

pre_median = np.median(pre_scores)
pre_lower = np.percentile(pre_scores, 2.5)
pre_upper = np.percentile(pre_scores, 97.5)

sen_median = np.median(sen_scores)
sen_lower = np.percentile(sen_scores, 2.5)
sen_upper = np.percentile(sen_scores, 97.5)

spe_median = np.median(spe_scores)
spe_lower = np.percentile(spe_scores, 2.5)
spe_upper = np.percentile(spe_scores, 97.5)

print(f"AUC: Median = {auc_median}, 95% CI = [{auc_lower}, {auc_upper}]")
print(f"F1: Median = {f1_median}, 95% CI = [{f1_lower}, {f1_upper}]")
print(f"ACC: Median = {acc_median}, 95% CI = [{acc_lower}, {acc_upper}]")
print(f"PRE: Median = {pre_median}, 95% CI = [{pre_lower}, {pre_upper}]")
print(f"SEN: Median = {sen_median}, 95% CI = [{sen_lower}, {sen_upper}]")
print(f"SPE: Median = {spe_median}, 95% CI = [{spe_lower}, {spe_upper}]")

In [56]:
results = pd.DataFrame({'Tree_auc':[0.6978,0.4262,0.6744,0.3359,0.5846,0.6976],
                'RF_auc':[0.6973,0.4472,0.6671,0.3435,0.6458,0.6745],
                'XGB_auc':[0.7175,0.4561,0.6541,0.3388,0.7010,0.6416],
                'LGB_auc':[0.7103,0.4545,0.6324,0.3295,0.7355,0.6048],
                'SVM_auc':[0.6979,0.2957,0.7670,0.4020,0.2357,0.9070],
                'NB_AUC':[0.7052,0.4561,0.6454,0.3357,0.7143,0.6275],
                'GBDT_AUC':[0.7253,0.4529,0.6874,0.3550,0.6233,0.7032]
                },index=['AUC','F1','ACC','PRE','SEN','SPE'])

In [ ]:
results_1 = results_1 / np.sqrt((results_1 ** 2).sum())

In [ ]:
def topsis(data, weight=None):
 
    Z = pd.DataFrame([data.max(), data.min()], index=['正理想解', '负理想解'])
 
    weight = entropyWeight(data) if weight is None else np.array(weight)
    Result = data.copy()
    Result['正理想解'] = np.sqrt(((data - Z.loc['正理想解']) ** 2 * weight).sum(axis=1))
    Result['负理想解'] = np.sqrt(((data - Z.loc['负理想解']) ** 2 * weight).sum(axis=1))
 
    Result['综合得分指数'] = Result['负理想解'] / (Result['负理想解'] + Result['正理想解'])  
    Result['排序'] = Result.rank(ascending=False)['综合得分指数']
 
    return Result, Z, weight
 
weight = [1/6,1/6,1/6,1/6,1/6,1/6,]
Result, Z, weight = topsis(results_1, weight)

In [125]:
def mult_model(data):

    result = pd.DataFrame()

    y_pred_tree = model_tree_result.predict(data)
    y_proba_tree = model_tree_result.predict_proba(data)[:,1]

    y_pred_rf = model_rf_result.predict(data)
    y_proba_rf = model_rf_result.predict_proba(data)[:,1]   

    y_pred_xgb = model_xgb_result.predict(data)
    y_proba_xgb = model_xgb_result.predict_proba(data)[:,1] 

    y_pred_lgb = model_lgb_result.predict(data)
    y_proba_lgb = model_lgb_result.predict_proba(data)[:,1] 

    y_pred_svm = model_svm_result.predict(data)
    y_proba_svm = model_svm_result.predict_proba(data)[:,1] 

    y_pred_nb = model_nb_result.predict(data)
    y_proba_nb = model_nb_result.predict_proba(data)[:,1]      

    y_pred_gbdt = model_gbdt_result.predict(data)
    y_proba_gbdt = model_gbdt_result.predict_proba(data)[:,1]
   
    result['y_probas'] = y_proba_tree*0.141+y_proba_rf*0.153+y_proba_xgb*0.155+y_proba_lgb*0.151+y_proba_svm*0.089+y_proba_nb*0.153+y_proba_gbdt*0.158
 
    result.loc[result['y_probas'] <= 0.5,'y_pred'] = np.int(0)
    result.loc[result['y_probas']  > 0.5,'y_pred'] = np.int(1)

    return result